## The forward and backward passes

In [1]:
#| default_exp backpropagation

In [2]:
#|export
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

# dataloader & standardization as a tensor
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## Foundations version

### Basic architecture

In [3]:
n,m = x_train.shape
c = y_train.max()+1 # add +1 due to indexing

# rows, columns and dependent variable
n,m,c 

(50000, 784, tensor(10))

In [4]:
# num hidden layer 
nh = 50

# Manual definition of a neural network

## Try different weight initialization practices here https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/ To do

In [5]:
#|export
# define weights and biases these are generating random numbers off a normal distribution
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [6]:
# A single layer
def lin(x, w, b): return x@w + b

In [7]:
t = lin(x_valid, w1, b1)
f'Shape of x_valid: rows & columns{t.shape}', f'Shape of w1: rows & columns {w1.shape}', f'Shape of b1: rows & columns {b1.shape}'

('Shape of x_valid: rows & columnstorch.Size([10000, 50])',
 'Shape of w1: rows & columns torch.Size([784, 50])',
 'Shape of b1: rows & columns torch.Size([50])')

Notice that the first and last dimensionality are retained.

In [8]:
def relu(x): return x.clamp_min(0.) 

def relu2deriv(output):
    return output > 0

# normal implementation
# def relu(x):
#     return (x > 0) * x

In [9]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [10]:
relu2deriv(t)

tensor([[False,  True, False,  ...,  True,  True,  True],
        [ True,  True, False,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True, False,  True],
        ...,
        [ True,  True, False,  ...,  True, False,  True],
        [ True, False,  True,  ..., False, False,  True],
        [ True, False,  True,  ..., False,  True,  True]])

In [11]:
def model(xb):
    "Multilayer perceptron: a single layer 3 layers"
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [12]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

### Loss function: MSE

(Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [13]:
res.shape,y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [14]:
(res-y_valid).shape

torch.Size([10000, 10000])

We need to get rid of that trailing (,1), in order to use `mse`.

In [15]:
res.shape, res[:,0].shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [16]:
res.squeeze().shape

torch.Size([10000])

In [17]:
(res[:,0]-y_valid).shape

torch.Size([10000])

In [18]:
y_train,y_valid = y_train.float(),y_valid.float()

preds = model(x_train)
preds.shape

torch.Size([50000, 1])

# mean squared error function 
find difference between predicted (output) and actual (target) square the result and find the mean. NB: a reduce function 

In [19]:
def mse(output, targ): return (output[:,0]-targ).pow(2).mean() 

In [20]:
mse(preds, y_train)

tensor(4308.76)

### Gradients and backward pass

In [21]:
!pip install sympy --q

In [22]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**2, x)

2*x

In [23]:
diff(3*x**2+9, x)

6*x

### How can you write the formula of this implementation with sympy? Todo

In [24]:
def lin_grad(inp, out, w, b):
    # grad/derivative of matmul with respect to input (enables us to update the weights)
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0) # constant

In [25]:
# using the python debugger
# def lin_grad(inp, out, w, b):
#     # grad/derivative of matmul with respect to input (enables us to update the weights)
#     inp.g = out.g @ w.t()
#     import pdb; pdb.set_trace()
#     w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
#     b.g = out.g.sum(0) # constant the bias

Review a tutorial on how to use pdb https://realpython.com/python-debugging-pdb/ todo

In [26]:
def forward_and_backward(inp, targ):
    # forward pass:
    # 3 layer NN: linear layer, Relu and linear output layer
    # loss MSE: (input - output) ** 2 reduce operation measure of success 
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    loss = res.pow(2).mean()
    
    # backward pass:
    # find the derivative with respect to input
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [27]:
forward_and_backward(x_train, y_train)

In [28]:
# Save for testing against later
def get_grad(x): return x.g.clone() # copies and saves the gradients
chks = w1,w2,b1,b2,x_train # 
grads = w1g,w2g,b1g,b2g,ig = map(get_grad, chks)

We cheat a little bit and use PyTorch autograd to check our results.

In [29]:
def mkgrad(x): return x.clone().requires_grad_(True) 
ptgrads = w12,w22,b12,b22,xt2 = map(mkgrad, chks)

In [30]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [31]:
loss = forward(xt2, y_train)
loss.backward()

In [32]:
for a,b in zip(grads, ptgrads): test_close(a.grad, b, eps=0.01)

## Refactor model

### Layers as classes

In [33]:
#|export
class Relu():
    """If the number is negative is to converted to 0
    
    Input: Output from the linear layer
    
    Adds an extra step of calculating the gradients with respect to the inputs.
    """
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [34]:
#|export
class Lin():
    """
    Linear layer component of a MLP
    inputs are the weights and biases randomly initiated.
    
    Backward is moving the delta signal around to adjust the loss.
    """
    
    def __init__(self, w, b): self.w,self.b = w,b

    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [35]:
#|export
class Mse():
    """
    Mean squared error this is the measure of success.
    
    input: the actual and prediction values from the linear layers
    backward stores the gradients and allows us to adjust the delta signal
    
    """
    def __call__(self, inp, targ):
        self.inp,self.targ = inp,targ
        self.out = mse(inp, targ)
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [36]:
#|export
class Model():
    """
    This is an object that combines the linear layer, relu and another linear layer (Making it a 3 layer MLP)
    input: weights, biases
    output: loss of the neural network
    
    """
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [37]:
model = Model(w1, b1, w2, b2)

In [38]:
loss = model(x_train, y_train)

In [39]:
#import pdb; pdb.set_trace()
model.backward()

In [40]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Module.forward()

In [41]:
#|export
class Module():
    """Compress representation"""
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [42]:
#|export
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [43]:
#|export
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [44]:
#|export
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [45]:
model = Model(w1, b1, w2, b2)

In [46]:
loss = model(x_train, y_train)

In [47]:
model.backward()

In [48]:
# import pdb; pdb.set_trace()
# model.backward()

In [49]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Autograd

In [50]:
#|export
# Pytorch
from torch import nn
import torch.nn.functional as F

In [51]:
#|export
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [52]:
#|export
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [53]:
#|export
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [54]:
import nbdev; nbdev.nbdev_export()

In [55]:
l0 = model.layers[0] # take the output of the first linear layer
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])

In [56]:
model.layers[2].w.grad

tensor([[-225.58],
        [ -10.21],
        [  -1.70],
        [ -83.60],
        [-101.32],
        [ -71.76],
        [ -25.83],
        [ -11.49],
        [-159.47],
        [-135.16],
        [-347.70],
        [ -61.26],
        [ -16.00],
        [ -53.31],
        [  -5.96],
        [-307.80],
        [ -85.18],
        [-331.73],
        [-143.83],
        [  -5.53],
        [-230.15],
        [ -72.28],
        [ -35.72],
        [-139.54],
        [ -77.20],
        [ -62.01],
        [   2.87],
        [-315.55],
        [  -9.66],
        [ -15.51],
        [-597.66],
        [-170.06],
        [-221.39],
        [-132.06],
        [-240.11],
        [-418.18],
        [ -57.13],
        [-250.77],
        [ -41.16],
        [ -58.84],
        [ -51.26],
        [-112.07],
        [-268.97],
        [ -27.62],
        [  -1.46],
        [-210.21],
        [  -1.33],
        [-168.76],
        [ -14.11],
        [ -50.50]])